In [1]:
import sqlalchemy as sa
from lagoon.db.connection import get_session
from lagoon.db import schema as sch
import numpy as np
import pandas as pd
import arrow

sess = get_session().__enter__()

## Entities and Observations Basics

In [3]:
entities = sess.query(sch.Entity) #this is a query object
print(entities.count())
# len(entities) is invalid, however, indexing works
print(entities[:3])

996703
[<lagoon.db.schema.Entity 1520737: EntityTypeEnum.person hans deragon <hans@deragon.biz>>, <lagoon.db.schema.Entity 1520738: EntityTypeEnum.person ombongi moraa fe <moraa.lovetakes2@gmail.com>>, <lagoon.db.schema.Entity 1520739: EntityTypeEnum.person kenneth love <klove@tax.ok.gov>>]


In [5]:
entities_list = entities.all() #the all() converts a query to a list
print(len(entities_list))
# NOTE: dealing with lists takes a lot of time, so stick to queries.
# NOTE: When iterating, i.e. `for entity in entities`, time taken is the same for a) queries and b) queries converted to lists using .all(). So, stick to queries.

996703


In [6]:
observations = sess.query(sch.Observation)
print(observations.count())

3975278


## Properties

In [7]:
e = entities[0]
for attr in dir(e):
    if not attr.startswith('__'):
        print(f"'{attr}': {getattr(e,attr)}")

'_sa_class_manager': <ClassManager of <class 'lagoon.db.schema.Entity'> at 1078aaf90>
'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x1046dd160>
'_sa_registry': <sqlalchemy.orm.decl_api.registry object at 0x10787e640>
'attrs': {'name': 'andrew barnert', 'email': 'abarnert@yahoo.com'}
'batch': Batch(id=25, resource='ocean-python.pck', ingest_time=datetime.datetime(2021, 8, 5, 23, 3, 10, 323383), revision=None)
'batch_id': 25
'fused': <bound method Entity.fused of <lagoon.db.schema.Entity 1509623: EntityTypeEnum.person andrew barnert <abarnert@yahoo.com>>>
'id': 1509623
'metadata': MetaData()
'name': andrew barnert <abarnert@yahoo.com>
'obs_as_dst': SELECT observation.id AS observation_id, observation.batch_id AS observation_batch_id, observation.type AS observation_type, observation.value AS observation_value, observation.time AS observation_time, observation.attrs AS observation_attrs, observation.dst_id AS observation_dst_id, observation.src_id AS observation_src

In [10]:
ob = observations[0]
for attr in dir(ob):
    if not attr.startswith('__'):
        print(f"'{attr}': {getattr(ob,attr)}")

'_sa_class_manager': <ClassManager of <class 'lagoon.db.schema.Observation'> at 1078ca8b0>
'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x14d706880>
'_sa_registry': <sqlalchemy.orm.decl_api.registry object at 0x10787e640>
'attrs': {}
'batch': Batch(id=25, resource='ocean-python.pck', ingest_time=datetime.datetime(2021, 8, 5, 23, 3, 10, 323383), revision=None)
'batch_id': 25
'dst': <lagoon.db.schema.Entity 1558842: EntityTypeEnum.message Message <1352951084.92039.YahooMailRC@web184706.mail.ne1.yahoo.com>>
'dst_id': 1558842
'id': 4093830
'metadata': MetaData()
'registry': <sqlalchemy.orm.decl_api.registry object at 0x10787e640>
'src': <lagoon.db.schema.Entity 1509623: EntityTypeEnum.person andrew barnert <abarnert@yahoo.com>>
'src_id': 1509623
'time': 2012-11-15 03:44:44
'type': ObservationTypeEnum.message_from
'value': None


## Connections

In [8]:
# Get all observations connected to this entity
# Note that the `obs_as_src` and `obs_as_dst` properties of an entity object are queries, so the best way to append one to the other is to concatenate the lists
obs = e.obs_as_src.all() + e.obs_as_dst.all() #this is a list
print(len(obs))

4329


In [9]:
# Get all other entities connected to ths entity
connected_e = []
ids = {e.id}
for ob in obs:
    ids.add(ob.src_id)
    ids.add(ob.dst_id)
ids.remove(e.id)
for id in ids:
    connected_e.append(sess.query(sch.Entity).get(id))
print(len(connected_e))

2552


## Fused Entities and Observations

In [4]:
## Get fused entities and observations
fused_entities = sess.query(sch.FusedEntity)
print(fused_entities.count())
fused_observations = sess.query(sch.FusedObservation)
print(fused_observations.count())

981865
3975266


In [17]:
fe = fused_entities[0]
for attr in dir(fe):
    if not attr.startswith('__'):
        try:
            print(f"'{attr}': {getattr(fe,attr)}")
        except:
            pass
print("_______________________________________________________")
fob = fused_observations[0]
for attr in dir(fob):
    if not attr.startswith('__'):
        print(f"'{attr}': {getattr(fob,attr)}")

'_sa_class_manager': <ClassManager of <class 'lagoon.db.schema_fused.FusedEntity'> at 1078dfd10>
'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x132469b80>
'_sa_registry': <sqlalchemy.orm.decl_api.registry object at 0x10787e640>
'attrs': {'name': 'tomer filiba', 'email': 'tomerfiliba@gmail.com'}
'batch_id': 25
'fusions': [EntityFusion(id_lowest=1509624, id_other=1529217, comment='email match: tomerfiliba@gmail.com'), EntityFusion(id_lowest=1509624, id_other=1510879, comment='email match: tomerfiliba@gmail.com'), EntityFusion(id_lowest=1509624, id_other=1509624, comment='email match: tomerfiliba@gmail.com')]
'id': 1509624
'metadata': MetaData()
'name': tomer filiba <tomerfiliba@gmail.com>
'obs_hops': <bound method FusedEntity.obs_hops of <lagoon.db.schema_fused.FusedEntity 1509624: EntityTypeEnum.person tomer filiba <tomerfiliba@gmail.com>>>
'registry': <sqlalchemy.orm.decl_api.registry object at 0x10787e640>
'type': EntityTypeEnum.person
__________________________

## Hops

In [2]:
# Get observations k hops out
some_entity = sess.query(sch.FusedEntity).get(1509628)
for k in [1,2,3]:
    print(len(some_entity.obs_hops(k))) #these are lists
print(some_entity.obs_hops(1)[0])

9
32
88
<lagoon.db.schema_fused.FusedObservation 4117269: (ObservationTypeEnum.message_from@2000-12-13, <lagoon.db.schema_fused.FusedEntity 1509628: EntityTypeEnum.person per kraulis <per@sbc.su.se>>, <lagoon.db.schema_fused.FusedEntity 1558117: EntityTypeEnum.message Message <mailman.976980782.10845.clpa-moderators@python.org>>)>


### Restrict by time

In [3]:
obs = some_entity.obs_hops(1, time_min=arrow.get('2000-01-01').datetime, time_max=arrow.get('2000-12-31').datetime)
print(len(obs))

7


In [6]:
# obs_hops is a better way instead of getting sources and destinations separately

commits = sess.query(sch.FusedEntity).where(sch.FusedEntity.type == sch.EntityTypeEnum.git_commit)
commit = commits[0]
# print(commit.obs_as_src).count() #this will NOT work since `'FusedEntity.obs_as_src' is not available due to lazy='raise'`
# Likewise for dst

# Instead, do this:
obs_as_src = sess.query(sch.FusedObservation).where(sch.FusedObservation.src == commit)
obs_as_dst = sess.query(sch.FusedObservation).where(sch.FusedObservation.dst == commit)
print(obs_as_src.count() + obs_as_dst.count())

# That should be equal to this
print(len(commit.obs_hops(1)))

# Since they are equivalent, it is better to just use obs_hops instead of obs_as_src and obs_as_dst

61
61


## Batches

In [5]:
batches = sess.query(sch.Batch).all()
print(batches)

[Batch(id=3, resource='ingest-git-github.com/python/cpython.git', ingest_time=datetime.datetime(2021, 7, 9, 17, 37, 25, 167273), revision=None), Batch(id=25, resource='ocean-python.pck', ingest_time=datetime.datetime(2021, 8, 5, 23, 3, 10, 323383), revision=None)]


In [12]:
# Check that all entities and observations belong to these batches
fe_batches = sess.query(sch.FusedEntity.batch_id).distinct()
print(fe_batches.all())
fo_batches = sess.query(sch.FusedObservation.batch_id).distinct()
print(fo_batches.all())

[(25,), (3,)]
[(3,), (25,)]


In [4]:
# Get number of entities and observations of each type in each batch
from lagoon.db.schema import EntityTypeEnum, ObservationTypeEnum
with get_session() as sess:
    for batch_id in [3,25]:
        print(f'Batch id {batch_id}:')
        for typ in EntityTypeEnum:
            entities = sess.query(sch.FusedEntity).where(sch.FusedEntity.batch_id==batch_id).where(sch.FusedEntity.type==typ)
            print(f'{typ}: {entities.count()}')
        for typ in ObservationTypeEnum:
            obs = sess.query(sch.FusedObservation).where(sch.FusedObservation.batch_id==batch_id).where(sch.FusedObservation.type==typ)
            print(f'{typ}: {obs.count()}')

Batch id 3:
EntityTypeEnum.git_commit: 128712
EntityTypeEnum.file: 18599
EntityTypeEnum.message: 0
EntityTypeEnum.person: 1638
ObservationTypeEnum.attached_to: 0
ObservationTypeEnum.committed: 128712
ObservationTypeEnum.created: 128712
ObservationTypeEnum.message_cc: 0
ObservationTypeEnum.message_from: 0
ObservationTypeEnum.message_ref: 0
ObservationTypeEnum.message_to: 0
ObservationTypeEnum.modified: 355288
Batch id 25:
EntityTypeEnum.git_commit: 0
EntityTypeEnum.file: 0
EntityTypeEnum.message: 799019
EntityTypeEnum.person: 33897
ObservationTypeEnum.attached_to: 0
ObservationTypeEnum.committed: 0
ObservationTypeEnum.created: 0
ObservationTypeEnum.message_cc: 0
ObservationTypeEnum.message_from: 977862
ObservationTypeEnum.message_ref: 2384689
ObservationTypeEnum.message_to: 3
ObservationTypeEnum.modified: 0


In [ ]:
# Check if there is batch overlap
for batch_id in [3,25]:
    print(f'Batch id {batch_id}:')
    entity_ids = set()
    obs = sess.query(sch.FusedObservation).where(sch.FusedObservation.batch_id==batch_id)
    for ob in obs:
        entity_ids.add(ob.src_id)
        entity_ids.add(ob.dst_id)
    for entity_id in entity_ids:
        entity = sess.query(sch.FusedEntity).get(entity_id)
        if entity.batch_id != batch_id:
            print(f'Mismatch: {entity.id}')

# RESULT: There is no overlap 

## Get start and end times in a batch
Alternatively, delete the batch portion to get this for the whole graph

In [3]:
obs = (
    sess.query(sch.FusedObservation)
    .where(sch.FusedObservation.batch_id == 3)
)
times = sorted([ob.time for ob in obs])
print(times[0])
print(times[-1])

1990-08-09 14:25:15
2021-06-28 22:02:18


In [4]:
obs = (
    sess.query(sch.FusedObservation)
    .where(sch.FusedObservation.batch_id == 25)
)
times = sorted([ob.time for ob in obs])
print(times[0])
print(times[-1])

1995-03-16 06:08:16
2021-05-01 03:07:50


## Toxicity analysis

In [4]:
# All attributes for all entities
keys = sess.query(sa.func.jsonb_object_keys(sch.FusedEntity.attrs)).distinct()
sorted(keys.all())

[('badwords_ex_googleInstantB_any',),
 ('badwords_ex_mrezvan94Harassment_Appearance',),
 ('badwords_ex_mrezvan94Harassment_Generic',),
 ('badwords_ex_mrezvan94Harassment_Intelligence',),
 ('badwords_ex_mrezvan94Harassment_Politics',),
 ('badwords_ex_mrezvan94Harassment_Racial',),
 ('badwords_ex_mrezvan94Harassment_Sexual',),
 ('badwords_ex_swearing_any',),
 ('body_text',),
 ('computed_badwords_googleInstantB_any',),
 ('computed_badwords_mrezvan94Harassment_Appearance',),
 ('computed_badwords_mrezvan94Harassment_Generic',),
 ('computed_badwords_mrezvan94Harassment_Intelligence',),
 ('computed_badwords_mrezvan94Harassment_Politics',),
 ('computed_badwords_mrezvan94Harassment_Racial',),
 ('computed_badwords_mrezvan94Harassment_Sexual',),
 ('computed_badwords_swearing_any',),
 ('email',),
 ('flagged_abuse',),
 ('message',),
 ('name',),
 ('origin_filename',),
 ('subject',)]

In [5]:
# All attributes for all message entities
# NOTE: Same as above, except for the absences of 'email' and 'name' (these are only for person presumably), and 'message' (only for commits presumably)
mkeys = sess.query(sa.func.jsonb_object_keys(sch.FusedEntity.attrs)).where(sch.FusedEntity.type == sch.EntityTypeEnum.message).distinct()
sorted(mkeys.all())

[('badwords_ex_googleInstantB_any',),
 ('badwords_ex_mrezvan94Harassment_Appearance',),
 ('badwords_ex_mrezvan94Harassment_Generic',),
 ('badwords_ex_mrezvan94Harassment_Intelligence',),
 ('badwords_ex_mrezvan94Harassment_Politics',),
 ('badwords_ex_mrezvan94Harassment_Racial',),
 ('badwords_ex_mrezvan94Harassment_Sexual',),
 ('badwords_ex_swearing_any',),
 ('body_text',),
 ('computed_badwords_googleInstantB_any',),
 ('computed_badwords_mrezvan94Harassment_Appearance',),
 ('computed_badwords_mrezvan94Harassment_Generic',),
 ('computed_badwords_mrezvan94Harassment_Intelligence',),
 ('computed_badwords_mrezvan94Harassment_Politics',),
 ('computed_badwords_mrezvan94Harassment_Racial',),
 ('computed_badwords_mrezvan94Harassment_Sexual',),
 ('computed_badwords_swearing_any',),
 ('flagged_abuse',),
 ('origin_filename',),
 ('subject',)]

In [9]:
# Check if flagged_abuse is ever not None
fused_entities = sess.query(sch.FusedEntity)
count = 0
for fe in fused_entities:
    if fe.attrs.get('flagged_abuse'):
        print(fe.attrs)
        print("______________________________________")
        count += 1
    if count==2:
        break

# RESULT: flagged_abuse is always None

## PEPs

In [7]:
# First check which mailing lists are included in OCEAN
messages = sess.query(sa.func.jsonb_extract_path_text(sch.FusedEntity.attrs,'origin_filename')).where(sch.FusedEntity.type == sch.EntityTypeEnum.message).distinct()
print([message[0] for message in messages.all()])

[None, '2016-09-mailman-python-ideas.mbox.gz', '1999-08-mailman-python-announce-list.mbox.gz', '2006-11-mailman-python-dev.mbox.gz', '2008-05-mailman-python-announce-list.mbox.gz', '2018-10-pipermail-python-announce-list.txt.gz', '2016-02-mailman-python-dev.mbox.gz', '2018-07-mailman-python-ideas.mbox.gz', '2007-01-mailman-python-announce-list.mbox.gz', '2006-08-pipermail-python-list.txt.gz', '2007-02-pipermail-python-ideas.txt.gz', '2014-02-pipermail-python-ideas.txt.gz', '2017-11-pipermail-python-list.txt.gz', '2003-06-pipermail-python-list.txt.gz', '2018-03-pipermail-python-list.txt.gz', '2006-07-pipermail-python-dev.txt.gz', '2005-05-pipermail-python-list.txt.gz', '2015-01-pipermail-python-announce-list.txt.gz', '2017-03-pipermail-python-dev.txt.gz', '2008-10-pipermail-python-dev.txt.gz', '2003-05-mailman-python-announce-list.mbox.gz', '2016-07-pipermail-python-list.txt.gz', '2018-05-pipermail-python-list.txt.gz', '2012-12-mailman-python-announce-list.mbox.gz', '2021-04-mailman-pyt

In [47]:
# Search relevant fields of messages and commits for mentions to "PEP XXX"
number = 1
search_string = rf'pep {number}\D' #must use lowercase since comparison is made using lowercase
links = sess.query(sch.Entity).where(sch.Entity.type == sch.EntityTypeEnum.message).where(sa.or_(sa.func.lower(sa.func.jsonb_extract_path_text(sch.Entity.attrs,field)).regexp_match(search_string) for field in ['subject','body_text']))
print(links.count())
links = sess.query(sch.FusedEntity).where(sch.FusedEntity.type == sch.EntityTypeEnum.git_commit).where(sa.func.lower(sa.func.jsonb_extract_path_text(sch.FusedEntity.attrs,'message')).contains(search_string))
print(links.count())

331
